In [2]:
import pandas as pd
import numpy as np
import os
import sys
import datetime

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Froms
from src.gcpUtils.auth import getCredentials
from src.gcpUtils.bigQuery import pandasToBq, tableToPandas
from src.gcpUtils.google_storage_manager import *


cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")

# Calcular dias de ruptura

# O que é computado por Filial × SKU
- Identificação de sequências (blocos)
    - Ordena por FILIAL, SKU, DATA.
    - Dentro de cada FILIAL+SKU detecta mudanças de estado (0->1 / 1->0) e numera blocos consecutivos.
    - Agrupa apenas as linhas com CONT_RUPTURA == 1 para obter cada bloco de ruptura.

- Cálculo de duração
    - Para cada bloco calcula data_inicial = min(DATA) e data_final = max(DATA).
    - dias_duracao = (data_final - data_inicial).days + 1.

- Identificador e ordenação
    - Atribui id_ruptura sequencial por FILIAL+SKU (1..n) conforme ordem cronológica.

- Status do período
    - Marca EM ANDAMENTO se data_final coincide com a última data observada para aquele FILIAL+SKU; caso contrário FINALIZADO.

In [11]:
query = """
SELECT
    FILIAL,
    SKU,
    DATA,
    CONT_RUPTURA,
    LEADTIME
FROM `planejamento-animale-292719.checklists_rollout.ANIMALE_checklist`
WHERE DATA > '2025-08-20' 
  AND DATA < '2025-09-01'
  AND FILIAL LIKE '%CM%'
ORDER BY FILIAL, SKU, DATA
"""

In [ ]:

def calcular_periodos_ruptura_com_gaps(df_input):
    df = df_input.copy()
    df['DATA'] = pd.to_datetime(df['DATA'])
    df = df.sort_values(by=['FILIAL', 'SKU', 'DATA'])
    
    # 0 = status manteve, 1 = status mudou
    df['mudou_status'] = df.groupby(['FILIAL', 'SKU'])['CONT_RUPTURA'].diff().fillna(0).ne(0).astype(int)
    
    df['bloco_id'] = df.groupby(['FILIAL', 'SKU'])['mudou_status'].cumsum()
    
    df_gaps = df[df['CONT_RUPTURA'] == 1].copy()
    
    if df_gaps.empty:
        return pd.DataFrame(columns=['FILIAL', 'SKU', 'id_ruptura', 'data_inicial', 'data_final', 'dias_duracao', 'status'])

    df_resumo = df_gaps.groupby(['FILIAL', 'SKU', 'bloco_id']).agg(
        data_inicial=('DATA', 'min'),
        data_final=('DATA', 'max')
    ).reset_index()
    
    df_resumo['dias_duracao'] = (df_resumo['data_final'] - df_resumo['data_inicial']).dt.days + 1
    
    df_resumo = df_resumo.sort_values(by=['FILIAL', 'SKU', 'data_inicial'])
    
    df_resumo['id_ruptura'] = df_resumo.groupby(['FILIAL', 'SKU']).cumcount() + 1
    
    max_data = df['DATA'].max()
    df_resumo['status'] = np.where(
        df_resumo['data_final'] == max_data, 
        'EM ANDAMENTO', 
        'FINALIZADO'
    )
    
    colunas_finais = [
        'FILIAL', 
        'SKU', 
        'id_ruptura', 
        'data_inicial', 
        'data_final', 
        'dias_duracao', 
        'status'
    ]
    
    return df_resumo[colunas_finais]


df_raw = tableToPandas(query, 'planejamento-animale-292719', cred)

print("Processando lógica de gaps e rupturas...")

df_final = calcular_periodos_ruptura_com_gaps(df_raw)

local_path = os.path.join(os.getcwd(), '../data')
if not os.path.exists(local_path):
    os.makedirs(local_path)

file_path = os.path.join(local_path, 'rupturas_animale_checklist.xlsx')
df_final.to_excel(file_path, index=False)

print(f"Processo concluído. Arquivo salvo em: {file_path}")
print(f"Total de períodos de ruptura identificados: {len(df_final)}")
display(df_final.head())

Processando lógica de gaps e rupturas...
Processo concluído. Arquivo salvo em: c:\Users\Leonardo Verissimo\repositorio_final\Project_AZZAS2154\notebooks\data\rupturas_animale_checklist.xlsx
Total de períodos de ruptura identificados: 32367


,FILIAL,SKU,id_ruptura,data_inicial,data_final,dias_duracao,status
0,ANIMALE ALPHAVILLE CM,03.01.1625-0005-TAM_3,1,2025-08-21,2025-08-31,11,EM ANDAMENTO
1,ANIMALE ALPHAVILLE CM,03.01.1704-0005-TAM_3,1,2025-08-21,2025-08-31,11,EM ANDAMENTO
2,ANIMALE ALPHAVILLE CM,03.01.1737-1531-TAM_1,1,2025-08-21,2025-08-31,11,EM ANDAMENTO
3,ANIMALE ALPHAVILLE CM,03.01.1746-10057-TAM_2,1,2025-08-21,2025-08-31,11,EM ANDAMENTO
4,ANIMALE ALPHAVILLE CM,03.01.1749-0005-TAM_2,1,2025-08-21,2025-08-31,11,EM ANDAMENTO
